In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = None

# data input
- mis fuentes:
  
https://www.kaggle.com/datasets/fcpercival/160k-spotify-songs-sorted?resource=download&select=data.csv

https://www.kaggle.com/datasets/asifsadmine/spotify-playlists-dataset

In [3]:
df = pd.read_csv('datos/data.csv')
df.head()

,id,name,artists,duration_ms,release_date,year,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,mode,key,popularity,explicit
0,0gNNToCW3qjabgTyBSjt3H,!Que Vida! - Mono Version,['Love'],220560,11/1/66,1966,0.5250,0.600,0.540,0.00305,0.100,-11.803,0.0328,125.898,0.547,1,9,26,0
1,0tMgFpOrXZR6irEOLNWwJL,"""40""",['U2'],157840,2/28/83,1983,0.2280,0.368,0.480,0.70700,0.159,-11.605,0.0306,150.166,0.338,1,8,21,0
2,2ZywW3VyVx6rrlrX75n3JB,"""40"" - Live",['U2'],226200,8/20/83,1983,0.0998,0.272,0.684,0.01450,0.946,-9.728,0.0505,143.079,0.279,1,8,41,0
3,6DdWA7D1o5TU2kXWyCLcch,"""40"" - Remastered 2008",['U2'],157667,2/28/83,1983,0.1850,0.371,0.545,0.58200,0.183,-9.315,0.0307,150.316,0.310,1,8,37,0
4,3vMmwsAiLDCfyc1jl76lQE,"""40"" - Remastered 2008",['U2'],157667,2/28/83,1983,0.1850,0.371,0.545,0.58200,0.183,-9.315,0.0307,150.316,0.310,1,8,35,0


mi dataset tiene los siguientes campos:
- `id` -- id único de cada canción
- `name` -- nombre de la canción 
- `artists` -- nombre del intérprete
- `duration_ms` -- campo numérico, indica la longitud de la canción en ms
- `release_date` -- fecha de publicación 
- `acousticness` 
- `danceability`
- `energy`
- `instrumentalness`
- `liveness`
- `loudness`
- `speechiness`
- `tempo`
- `valence`
- `mode`
- `key` -- tono en el que está la canción
- `popularity` -- popularidad de la canción

In [7]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
duration_ms,169907.0,231407.085988,121322.336392,5108.0,171040.0000,208600.000000,262966.5000,5403500.000
year,169907.0,1977.223234,25.593318,1921.0,1957.0000,1978.000000,1999.0000,2020.000
acousticness,169907.0,0.493217,0.376628,0.0,0.0945,0.492000,0.8880,0.996
danceability,169907.0,0.538147,0.175345,0.0,0.4170,0.548000,0.6670,0.988
energy,169907.0,0.488591,0.267391,0.0,0.2630,0.481000,0.7100,1.000
instrumentalness,169907.0,0.161939,0.309330,0.0,0.0000,0.000204,0.0868,1.000
liveness,169907.0,0.206692,0.176797,0.0,0.0984,0.135000,0.2630,1.000
loudness,169907.0,-11.370311,5.666795,-60.0,-14.4700,-10.474000,-7.1180,3.855
speechiness,169907.0,0.094058,0.149938,0.0,0.0349,0.045000,0.0754,0.969
tempo,169907.0,116.947850,30.727079,0.0,93.5160,114.777000,135.7120,244.091


In [9]:
df.describe(include='object').T

,count,unique,top,freq
id,169907,169907,0gNNToCW3qjabgTyBSjt3H,1
name,169907,132938,Summertime,62
artists,169907,33375,['Эрнест Хемингуэй'],1215
release_date,169907,10882,1945,1449
